In [313]:
import numpy as np
import torch
import torch.nn.functional as F

In [314]:
data = open('names.txt').read().splitlines()
data[:5]

['emma', 'olivia', 'ava', 'isabella', 'sophia']

In [315]:
chars = sorted(list({ch for w in data for ch in w}))

# First try for trigram

In [145]:
# Used 27*27=729 input dimension for input of two characters
# doesnt work well though ... try to think why this isnt good. Drops information?

In [316]:
chars = sorted(list({ch for w in data for ch in w}))
chars = ['.'] + chars
ytoi = {s:i for i,s in enumerate(chars)}
ytos = {i:s for s,i in ytoi.items()}
charcount=len(stoi)

In [317]:
xtoi = {}
i = 0
for ch1 in chars:
    for ch2 in chars: 
        xtoi[(ch1, ch2)] = i
        i += 1
        
xtos = {i:s for s,i in xtoi.items()}
xdim = len(xtoi)

In [320]:
# build a trigram model

# create the training set of bigrams
xs, ys = [], []
for w in data:
    chs = ['.'] + ['.'] + list(w) + ['.'] + ['.']
    for ch1, ch2, ch3 in zip(chs, chs[1:], chs[2:]):
        ix = xtoi[(ch1, ch2)]
        iy = ytoi[ch3]
        xs.append(ix)
        ys.append(iy)
xs = torch.tensor(xs)
ys = torch.tensor(ys)
num = xs.shape[0]
print('number of examples: ', num)

# randomly initialise 27 neurons' weights. each neuron receives 27 inputs
g = torch.Generator().manual_seed(2147483647)
W = torch.randn((xdim, 27), generator=g, requires_grad=True)

number of examples:  260179


In [321]:
# print([itos[x[0].item()] + itos[x[1].item()] for x in xs[:6]])
print([xtos[x.item()] for x in xs[:6]])
print([ytos[y.item()] for y in ys[:6]])

[('.', '.'), ('.', 'e'), ('e', 'm'), ('m', 'm'), ('m', 'a'), ('a', '.')]
['e', 'm', 'm', 'a', '.', '.']


In [322]:
# gradient descend
for k in range(100):
    # forward pass
    xenc = F.one_hot(xs, num_classes=xdim).float() # input to the network: one_hot encoding
    logits = xenc @ W # predict log-counts
    counts = logits.exp() # counts, equivalent to N
    probs = counts / counts.sum(1, keepdims=True) # probabilites for next character
    loss = -probs[torch.arange(num), ys].log().mean()
    print(loss.item())
    
    # backward pass
    W.grad = None # set the gradients to zero
    loss.backward()
    
    # update
    W.data += -50 * W.grad

3.7422749996185303
3.5306496620178223
3.3819174766540527
3.2758371829986572
3.1978492736816406
3.1354622840881348
3.082799196243286
3.0368664264678955
2.995882987976074
2.958749294281006
2.924757480621338
2.8934175968170166
2.8643672466278076
2.8373255729675293
2.8120646476745605
2.788397789001465
2.7661678791046143
2.745241641998291
2.725506067276001
2.706862449645996
2.689223289489746
2.6725120544433594
2.656658887863159
2.6416001319885254
2.627277374267578
2.613636016845703
2.6006274223327637
2.5882060527801514
2.576331377029419
2.5649654865264893
2.554074287414551
2.543628215789795
2.533597946166992
2.5239593982696533
2.514688491821289
2.5057642459869385
2.497166872024536
2.488878011703491
2.4808812141418457
2.4731600284576416
2.465700149536133
2.4584872722625732
2.4515087604522705
2.4447524547576904
2.438206672668457
2.4318606853485107
2.4257047176361084
2.419728994369507
2.4139249324798584
2.4082834720611572
2.4027976989746094
2.3974592685699463
2.3922622203826904
2.3871996402740

In [144]:
# Sample from the neutral net

g = torch.Generator().manual_seed(2147483647)

for i in range(5):
    out=[]
    ix = 0
    while True:
        # p = P[ix]
        xenc = F.one_hot(torch.tensor([ix]), num_classes=xdim).float()
        logits = xenc @ W # predict log-counts
        counts = logits.exp() # counts, equivalent to N
        p = counts / counts.sum(1, keepdims=True)
        
        ix = torch.multinomial(p, num_samples=1, replacement=True, generator=g).item()
        out.append(ytos[ix])
        if ix == 0: 
            break
    print(''.join(out))

emracvvaromkxttszpgschuhqvbmxemc.
emmxqvydusmasemripdkfbllvkfemhusxnumhtdjixwlu.
embw.
emhpyucemhqhrxktpdiqslu.
emajadkrlqriabwtvscjmhqwtpxemygdwfvnnjcjchwzwjywtogdnppollwtqyhdmwdjuyggluyyyqwvbuytofbcvbmbotdkajlnplpnpjmjmaoodqy.


# Second try for trigram

In [ ]:
# Not sure if this is a trigram model. Not taking order into account here. 
# a b -> c    and   b a -> c      is the same

In [333]:
chars = sorted(list({ch for w in data for ch in w}))
chars = ['.'] + chars
stoi = {s:i for i,s in enumerate(chars)}
itos = {i:s for s,i in stoi.items()}
numchars=len(chars)

In [335]:
# stoi

In [389]:
# build a trigram model

# create the training set of bigrams
xs, ys = [], []
for w in data:
    chs = ['.'] + ['.'] + list(w) + ['.']   # 2 start tokens for trigram. Only 1 end token needed.
    for ch1, ch2, ch3 in zip(chs, chs[1:], chs[2:]):
        ix1 = stoi[ch1]
        ix2 = stoi[ch2]
        iy = ytoi[ch3]
        xs.append((ix1, ix2))
        ys.append(iy)
xs = torch.tensor(xs)
ys = torch.tensor(ys)
num = xs.shape[0]
print('number of examples: ', num)

# randomly initialise 27 neurons' weights. each neuron receives 27 inputs
g = torch.Generator().manual_seed(2147483647)
W = torch.randn((27*2, 27), generator=g, requires_grad=True)

number of examples:  228146


In [390]:
print(xs[:6])
print(ys[:6])
print([itos[x[0].item()] + itos[x[1].item()] for x in xs[:6]])
print([itos[y.item()] for y in ys[:6]])

tensor([[ 0,  0],
        [ 0,  5],
        [ 5, 13],
        [13, 13],
        [13,  1],
        [ 0,  0]])
tensor([ 5, 13, 13,  1,  0, 15])
['..', '.e', 'em', 'mm', 'ma', '..']
['e', 'm', 'm', 'a', '.', 'o']


In [377]:
# Check reshaping ... 
#xenc = F.one_hot(torch.tensor([0, 0]), num_classes=3).float()
#xenc.view(-1, 2*3)

In [393]:
# gradient descend
for k in range(100):
    # forward pass
    xenc = F.one_hot(xs, num_classes=numchars).float() # input to the network: one_hot encoding
    logits = xenc.view(-1, 2*27) @ W # predict log-counts
    counts = logits.exp() # counts, equivalent to N
    probs = counts / counts.sum(1, keepdims=True) # probabilites for next character
    loss = -probs[torch.arange(num), ys].log().mean()
    print(loss.item())
    
    # backward pass
    W.grad = None # set the gradients to zero
    loss.backward()
    
    # update
    W.data += -50 * W.grad

2.3706207275390625
2.3684327602386475
2.3665053844451904
2.3648037910461426
2.3632960319519043
2.361957311630249
2.3607656955718994
2.3597023487091064
2.358751058578491
2.3578991889953613
2.357133626937866
2.3564453125
2.3558244705200195
2.3552639484405518
2.3547568321228027
2.354297637939453
2.3538804054260254
2.3535008430480957
2.3531551361083984
2.352839946746826
2.3525519371032715
2.352288007736206
2.352046489715576
2.3518245220184326
2.3516201972961426
2.3514323234558105
2.3512589931488037
2.3510985374450684
2.350950241088867
2.3508129119873047
2.3506855964660645
2.3505666255950928
2.3504562377929688
2.3503530025482178
2.350257158279419
2.3501670360565186
2.3500823974609375
2.350003480911255
2.349928855895996
2.3498589992523193
2.349792957305908
2.3497304916381836
2.3496718406677246
2.349616050720215
2.3495633602142334
2.349513530731201
2.349465847015381
2.3494205474853516
2.3493776321411133
2.349336624145508
2.349297046661377
2.349259853363037
2.349224090576172
2.3491897583007812

In [394]:
# Sample from the neutral net

g = torch.Generator().manual_seed(2147483647)

for i in range(10):
    out=[]
    ix1 = 0
    ix2 = 0
    while True:
        # p = P[ix]
        xenc = F.one_hot(torch.tensor([ix1, ix2]), num_classes=numchars).float()
        logits = xenc.view(-1, 2*27) @ W # predict log-counts
        counts = logits.exp() # counts, equivalent to N
        p = counts / counts.sum(1, keepdims=True)
        
        iy = torch.multinomial(p, num_samples=1, replacement=True, generator=g).item()
        out.append(ytos[iy])
        if iy == 0: 
            break
            
        ix1 = ix2
        ix2 = iy
    print(''.join(out))

mor.
ays.
merhurroydco.
konamaloe.
caneithizarie.
ok.
ondalaek.
shalekirierielah.
yshi.
haat.


# Third try for trigram

In [407]:
# Taking order into account ... 
# a b -> c    and   b a -> c      are note the same

In [420]:
chars = sorted(list({ch for w in data for ch in w}))
chars = ['.'] + chars
stoi = {s:i for i,s in enumerate(chars)}
itos = {i:s for s,i in stoi.items()}
numchars=len(chars)

In [419]:
xtoi = {}
i = 0
for ch1 in chars:
    for ch2 in chars: 
        xtoi[(ch1, ch2)] = i
        i += 1
        
xtos = {i:s for s,i in xtoi.items()}
xdim = len(xtoi)

In [455]:
# build a trigram model

# create the training set of bigrams
xs, ys = [], []
for w in data:
    chs = ['.'] + ['.'] + list(w) + ['.']   # 2 start tokens for trigram. Only 1 end token needed.
    for ch1, ch2, ch3 in zip(chs, chs[1:], chs[2:]):
        ix1 = stoi[ch1]
        ix2 = stoi[ch2]
        iy = ytoi[ch3]
        xs.append((ix1, ix2))
        ys.append(iy)
xs = torch.tensor(xs)
ys = torch.tensor(ys)
num = xs.shape[0]
print('number of examples: ', num)

# randomly initialise 27 neurons' weights. each neuron receives 27 inputs
g = torch.Generator().manual_seed(2147483647)
W = torch.randn((27,27,27), generator=g, requires_grad=True)

number of examples:  228146


In [410]:
print(xs[:6])
print(ys[:6])
print([itos[x[0].item()] + itos[x[1].item()] for x in xs[:6]])
print([itos[y.item()] for y in ys[:6]])

tensor([[ 0,  0],
        [ 0,  5],
        [ 5, 13],
        [13, 13],
        [13,  1],
        [ 0,  0]])
tensor([ 5, 13, 13,  1,  0, 15])
['..', '.e', 'em', 'mm', 'ma', '..']
['e', 'm', 'm', 'a', '.', 'o']


In [462]:
for i,row in enumerate(xs[:3]):
    print(i, row)

0 tensor([0, 0])
1 tensor([0, 5])
2 tensor([ 5, 13])


In [463]:
def one_hot_2d(data, num_classes=27):
    rows = data.shape[0]
    xenc = torch.zeros(rows, num_classes,num_classes).float()

    for i,row in enumerate(data):
        col1 = row[0].item()
        col2 = row[1].item()
        xenc[i, col1, col2] += 1.0
        
    return xenc

In [439]:
# import matplotlib.pyplot as plt
# %matplotlib inline

# plt.imshow(xenc)

In [471]:
W.shape

torch.Size([27, 27, 27])

In [493]:
# gradient descend
xenc = one_hot_2d(xs, num_classes=27)

for k in range(10):
    # forward pass
    #xenc = F.one_hot(xs, num_classes=27*27).float() # input to the network: one_hot encoding
    logits = xenc.view(-1, 27*27) @ W.view(27*27, -1) # predict log-counts
    counts = logits.exp() # counts, equivalent to N
    probs = counts / counts.sum(1, keepdims=True) # probabilites for next character
    loss = -probs[torch.arange(num), ys].log().mean()
    print(loss.item())
    
    # backward pass
    W.grad = None # set the gradients to zero
    loss.backward()
    
    # update
    W.data += -100 * W.grad

2.350231409072876
2.349199056625366
2.348179817199707
2.3471734523773193
2.3461804389953613
2.3451998233795166
2.344231605529785
2.343275308609009
2.3423309326171875
2.341398239135742


In [517]:
# Sample from the neutral net
g = torch.Generator().manual_seed(2147483647)

for i in range(10):
    out=[]
    ix1 = 0
    ix2 = 0
    while True:
        # p = P[ix]
        # xenc = F.one_hot(torch.tensor([ix1, ix2]), num_classes=27*27).float()
        xenc = one_hot_2d(torch.tensor([[ix1, ix2]]), num_classes=27)
        logits = xenc.view(-1, 27*27) @ W.view(27*27, -1) # predict log-counts
        counts = logits.exp() # counts, equivalent to N
        p = counts / counts.sum(1, keepdims=True)
        
        iy = torch.multinomial(p, num_samples=1, replacement=True, generator=g).item()
        out.append(ytos[iy])
        if iy == 0: 
            break
            
        ix1 = ix2
        ix2 = iy
    print(''.join(out))

mir.
axx.
mereyannyabraisemmaloah.
aqorwgtzarle.
oderaleice.
dmne.
deoosesona.
elyn.
r.
gagv.
